In [1]:
import pandas as pd

# Read the test data
test_defog = pd.read_csv('test/defog/02ab235146.csv')
# create a new column for the id which is the filename followed by '_' and the Time
test_defog['Id'] = '003f117e14' + '_' + test_defog['Time'].astype(str)
test_defog['Source'] = 'defog'

test_tdcsfog = pd.read_csv('test/tdcsfog/003f117e14.csv')
# create a new column for the id which is '003f117e14' followed by '_' and the Time
test_tdcsfog['Id'] = '003f117e14' + '_' + test_tdcsfog['Time'].astype(str)
test_tdcsfog['Source'] = 'tdcsfog'

# Combine the test data
test_data = pd.concat([test_defog, test_tdcsfog], ignore_index=False)

In [3]:

def unify_sampling_rate(data):
    # Interpolate defog and notype data from 100Hz to 128Hz
    def interpolate_data(df):
        df['Time'] = pd.to_timedelta(df['Time'], unit='ms')  # Convert 'Time' column to timedelta
        df_resampled = df.set_index('Time').resample('7.8125ms').interpolate(method='linear').reset_index()
        return df_resampled

    data_defog = data[data['Source'].isin(['defog', 'notype'])].groupby('SeriesId').apply(interpolate_data).reset_index(drop=True)
    data_tdcsfog = data[data['Source'] == 'tdcsfog']

    unified_data = pd.concat([data_tdcsfog, data_defog], ignore_index=True)
    return unified_data


def unify_acceleration_units(data):
    g_to_ms2 = 9.81

    data.loc[data['Source'].isin(['defog', 'notype']), ['AccV', 'AccML', 'AccAP']] *= g_to_ms2
    return data

def filter_data(data):
    filtered_data = data[(data['Valid'].fillna(True)) & (data['Task'].fillna(True))].reset_index(drop=True)
    return filtered_data

# test_data = unify_sampling_rate(test_data)
test_data = unify_acceleration_units(test_data)
test_data['Unnamed: 0'] = test_data['Time']
# filtered_test_data = filter_data(test_data)

In [6]:
test_data.head()

,Time,AccV,AccML,AccAP,Id,Source,Unnamed: 0
0,0,-8.972738,-2.951349,2.924906,003f117e14_0,defog,0
1,1,-8.969944,-2.958421,2.926564,003f117e14_1,defog,1
2,2,-7.647290,-3.366763,2.704516,003f117e14_2,defog,2
3,3,-9.742921,-2.651460,3.097750,003f117e14_3,defog,3
4,4,-9.307401,-3.019995,2.915445,003f117e14_4,defog,4


In [5]:
from autogluon.tabular import TabularPredictor

# Load the saved models
predictor_start_hes = TabularPredictor.load('AutogluonModels/start_hes', 
                                            require_version_match=False,
                                            require_py_version_match=False)
predictor_turn = TabularPredictor.load('AutogluonModels/turn', 
                                       require_version_match=False,
                                       require_py_version_match=False)
predictor_walking = TabularPredictor.load('AutogluonModels/walking', 
                                          require_version_match=False,
                                          require_py_version_match=False)

# Make predictions on the test data for each target event type
predictions = []
labels = ['StartHesitation', 'Turn', 'Walking']
predictors = {
    'StartHesitation': predictor_start_hes,
    'Turn': predictor_turn,
    'Walking': predictor_walking
}

for label in labels:
    print(f"Predicting probabilities for {label}...")
    predictor = predictors[label]
    predictions.append(predictor.predict(test_data), model='WeightedEnsemble_L2')

# Combine the predictions into a single DataFrame
predictions_df = pd.concat(predictions, axis=1)
predictions_df.columns = labels
print(predictions_df)



############################## WARNING ##############################
	Predictor Version: 0.6.2
	Current Version:   0.7.0
############################## WARNING ##############################

Found 3 mismatches between original and current metadata:

############################## WARNING ##############################
	Predictor Version: 0.6.2
	Current Version:   0.7.0
############################## WARNING ##############################

Found 3 mismatches between original and current metadata:

############################## WARNING ##############################
	Predictor Version: 0.6.2
	Current Version:   0.7.0
############################## WARNING ##############################

Found 3 mismatches between original and current metadata:


Predicting probabilities for StartHesitation...


ModuleNotFoundError: No module named 'sklearnex'